In [1]:
!pip install sec-api
import pandas as pd
from sec_api import XbrlApi
from sec_api import QueryApi
import pprint 
api_key = 'd3f7f3ef133011afaf4660819cb086ce63b4d74c654466add24862a521e6590b'
query_api = QueryApi(api_key)
xbrlApi = XbrlApi(api_key)

In [2]:
def fetch_all_filings_for_a_company(cik, form):
    all_filings = []
    from_ = 0
    size = 200 

    while True:
        query_params = {
            'query': {
                'query_string': {
                    'query': f'({form}) AND cik:"{cik}"'
                }
            },
            'from': from_,
            'size': size
        }
        
        response = query_api.get_filings(query_params)
        
        if 'filings' in response:
            filings_data = response['filings']
            all_filings.extend(filings_data)

            # Check if there is more data
            if len(filings_data) < size:
                break  # No more to fetch

            from_ += size
        else:
            print(f"Failed to fetch filings for {cik} .")
            break

    return all_filings


In [3]:
# Function to read the file and extract codes
def extract_codes_from_file(file_path):
    codes = []
    with open(file_path, 'r') as file:
        for line in file:
            # Split the line by whitespace and get the second part (the code)
            code = line.split()[1]
            # Convert the code to integer and add to the list
            codes.append(int(code))
    return codes

# Specify the path to the tiker.txt file
file_path = file_path = r'C:\Users\omarn\Desktop\PFE\ticker.txt'

# Extract the codes
codes_list = extract_codes_from_file(file_path)

print(codes_list)


[320193, 789019, 1067983, 731766, 200406, 1403161, 1046179, 34088, 104169, 884394, 19617, 80424, 1045810, 824046, 1141391, 93410, 1318605, 59478, 354950, 70858, 78003, 21344, 1551152, 1577552, 353278, 77476, 909832, 310158, 97745, 1730168, 313616, 1744489, 1094517, 63908, 937966, 1341439, 1800, 858877, 1114448, 901832, 1283699, 72971, 1467373, 732712, 753308, 320187, 1090727, 14272, 895421, 1166691, 1108524, 1163165, 97476, 796343, 1413329, 1707925, 100885, 804328, 1000275, 316709, 64803, 811809, 879764, 60667, 318154, 101829, 1350102, 1089113, 1144967, 1613103, 773840, 947263, 4962, 732717, 896878, 50863, 1140625, 217410, 1067839, 936468, 1633917, 51143, 1053507, 886982, 315189, 829224, 1065280, 313807, 1121404, 835403, 1000184, 18230, 8670, 887028, 831001, 1739940, 1364742, 1549802, 2488, 1274152, 1303523, 313838, 863064, 1001250, 12927, 1373715, 1119639, 1045609, 1668717, 1737806, 310764, 895728, 16868, 64040, 1103982, 896159, 92122, 1326160, 882095, 1103838, 1075531, 1559720, 62709

In [4]:
import pandas as pd
import numpy as np

def extract_financial_data(data):
    financial_store = {}

    for usGaapItem, facts in data.items():
        values = []
        indices = []

        for fact in facts:
            if isinstance(fact, dict):
                period = fact.get('period', {})
                if 'startDate' in period and 'endDate' in period:
                    index = period['startDate'] + '-' + period['endDate']
                elif 'instant' in period:
                    index = period['instant']
                else:
                    continue

                if index not in indices:
                    values.append(fact.get('value', np.nan))
                    indices.append(index)
            # Skipping invalid fact formats

        if values:
            financial_store[usGaapItem] = pd.Series(values, index=indices)

    financial_df = pd.DataFrame(financial_store)
    return financial_df.T


In [5]:
ten = fetch_all_filings_for_a_company(1562401,'formType:"10-K" OR formType:"10-Q"')
df_10 = pd.DataFrame(ten)
#df_10_cleaned = df_10[df_10['dataFiles'].map(lambda x: len(x) > 0)] #Nchoufou  idha lazemna na3mlou excepton kif mayla9ach dataFile
df_10.size

874

In [11]:
eight = fetch_all_filings_for_a_company(1353538,'formType:"8-K"')
df_8 = pd.DataFrame(eight)

In [17]:
df_8['periodOfReport']

0     2024-07-02
1     2024-06-17
2     2024-04-30
3     2024-04-19
4     2024-03-28
5     2024-01-11
6     2023-11-14
7     2023-10-30
8     2023-10-19
9     2023-10-11
10    2023-09-18
11    2023-08-31
12    2023-08-21
13    2023-07-20
14    2023-06-09
15    2023-05-12
16    2023-03-31
17    2023-02-01
18    2023-01-09
19    2023-01-04
20    2022-11-08
21    2022-08-11
22    2022-07-21
23    2022-05-11
24    2022-04-26
25    2022-03-30
26    2022-02-09
27    2022-01-01
28    2021-11-10
29    2021-10-12
30    2021-10-12
31    2021-02-08
32    2020-08-18
33    2019-11-27
34    2018-12-04
35    2018-08-27
36    2017-02-14
37    2016-07-15
38    2016-04-01
39    2015-11-18
40    2015-10-30
41    2014-07-25
42    2013-05-14
43    2013-05-06
44    2012-10-22
45    2009-04-03
46    2007-08-07
Name: periodOfReport, dtype: object

In [7]:
list_of_items= eight[0]['items']
substring = "Item 1.03"

In [8]:
##'Item 1.03' in eight[0]['items']
exists = any(substring in s for s in list_of_strings)
print(exists)

True


In [9]:
def check_Banccurapcy(list_items, item):
    exists = any(item in s for s in list_items)
    return 1 if exists else 0


In [10]:
x = check_Banccurapcy(list_of_items,substring)
print(x)

1


In [31]:
htm_url = df_10_cleaned['linkToHtml'][4]
xbrlJson = xbrlApi.xbrl_to_json(htm_url=htm_url)

In [11]:
pprint.pp(xbrlJson['BalanceSheets'])


{'CashAndCashEquivalentsAtCarryingValue': [{'decimals': '-6',
                                            'unitRef': 'U_USD',
                                            'period': {'instant': '2023-03-31'},
                                            'segment': {'dimension': 'srt:ProductOrServiceAxis',
                                                        'value': 'brka:InsuranceAndOtherMember'},
                                            'value': '23805000000'},
                                           {'decimals': '-6',
                                            'unitRef': 'U_USD',
                                            'period': {'instant': '2022-12-31'},
                                            'segment': {'dimension': 'srt:ProductOrServiceAxis',
                                                        'value': 'brka:InsuranceAndOtherMember'},
                                            'value': '32260000000'},
                                           {'decimals': '-

In [79]:
def clean_json_data(data):
    cleaned_data = {}

    for key, facts in data.items():
        cleaned_facts = [fact for fact in facts if 'segment' not in fact]
        if cleaned_facts:
            cleaned_data[key] = cleaned_facts

    return cleaned_data

In [13]:
cleandata = clean_json_data(xbrlJson['BalanceSheets'])

In [ ]:
cleandata

In [ ]:
balance_sheet_df = extract_financial_data(cleandata)
balance_sheet_df
#last_column_BS = balance_sheet_df.columns[-1]
#balance_sheet_df[last_column_BS].get('Assets', np.nan)
#last_column_BS

In [80]:
def clean_Financial_data(statement):
    for column in statement:

        # column has more than 5 NaN values
        is_nan_column = statement[column].isna().sum() > 5

        if column.endswith('_left') or column == 'key_0' or is_nan_column:
            statement = statement.drop(column, axis=1)
    
    # rearrange columns so that first column represents first quarter
    # e.g. 2018, 2019, 2020 - and not 2020, 2019, 2018
    sorted_columns = sorted(statement.columns.values)
    
    return statement[sorted_columns]

In [81]:
data = {
    "cik": None,
    "companyName": None,
    "ticker": None,
    "accessionNo": None,
    "Assets": None,
    "fy": None,
    "fp": None,
    "form": None,
    "filed": None,
    "Current_Assets": None,
    "Current_liabilities": None,
    "Stockholder_Equity": None,
    "Liabilities_And_StockholderEquity": None,
    "Earning_Before_Interest_And_Taxes": None,
    "Retained_Earnings": None,
    "Revenues": None,
    "Working_capital": None,
    "Liabilities": None,
    "NetCash_OperatingActivities":None,
    "NetCash_InvestingActivities": None,
    "NetCash_FinancingActivities": None,
    "Cash": None,
    "AccountsReceivable":None,
    "Inventory": None,
    "Current_Other_Assets": None,
    "Noncurrent_Assets": None,
    "Intangible_Assets": None,
    "AccountsPayable": None,
    "NetIncome": None,
    "GrossProfit": None,
    "Operating_Expenses": None,
    "Nonoperating_Income": None,
    "InterestExpense": None,
    "ShortTerm_Debt": None,
    "LongTerm_Debt": None,
    "Noncurrent_Liabilities": None,
   
}

In [83]:
import numpy as np
import pandas as pd

# Initialize the CSV file
csv_file = 'tst1.csv'
columns = ["cik", "companyName", "ticker", "accessionNo", "Assets","fy","fp","form","filed","Current_Assets","Current_liabilities","Stockholder_Equity","Liabilities_And_StockholderEquity","Earning_Before_Interest_And_Taxes","Retained_Earnings","Revenues","Working_capital","Liabilities","NetCash_OperatingActivities","NetCash_InvestingActivities","NetCash_FinancingActivities","Cash","AccountsReceivable","Inventory", 
           "Current_Other_Assets", "Noncurrent_Assets", "Intangible_Assets", 
           "AccountsPayable", "NetIncome", "GrossProfit", "Operating_Expenses", 
           "Nonoperating_Income", "InterestExpense", "ShortTerm_Debt", "LongTerm_Debt", 
           "Noncurrent_Liabilities"]

# Create an empty DataFrame with the desired columns and save it to the CSV file
pd.DataFrame(columns=columns).to_csv(csv_file, index=False)

# List of CIKs to process
cikss = [1068851]
limit = 10
def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return np.nan

for cik in cikss:
    ten = fetch_all_filings_for_a_company(cik, 'formType:"10-K" OR formType:"10-Q"')
    df_10 = pd.DataFrame(ten)
    eight = fetch_all_filings_for_a_company(cik,'formType:"8-K"')
    
    # Check if the DataFrame is empty
    if df_10.empty:
        print(f"No filings found for CIK {cik}. Skipping to the next CIK.")
        continue
    
    # Filter rows where 'dataFiles' key is present and non-empty
    df_10_cleaned = df_10[df_10['dataFiles'].map(lambda x: len(x) > 0 if isinstance(x, list) else False)]
    
    for index, row in df_10_cleaned.head(limit).iterrows():
        try:
            htm_url = row['linkToHtml']
            xbrlJson = xbrlApi.xbrl_to_json(htm_url=htm_url)
            
            # Clean and extract financial data
            cleaned_data = clean_json_data(xbrlJson['BalanceSheets'])
            cleaned_data_2 = clean_json_data(xbrlJson['StatementsOfCashFlows'])
            cleaned_data_3 = clean_json_data(xbrlJson['StatementsOfIncome'])
            financial_data = extract_financial_data(cleaned_data)
            financial_data_2 = extract_financial_data(cleaned_data_2)
            financial_data_3 = extract_financial_data(cleaned_data_3)
            balance_sheet = clean_Financial_data(financial_data)
            cash_flows = clean_Financial_data(financial_data_2)
            income_statement = clean_Financial_data(financial_data_3)
            last_column_BS = balance_sheet.columns[-1]
            last_column_CF = cash_flows.columns[-1]
            last_column_IS = income_statement.columns[-1]
            
            # Prepare the data to append to CSV
            data = {
                "cik": row['cik'],
                "companyName": row['companyName'],
                "ticker": row['ticker'],
                "accessionNo": row['accessionNo'],
                "Assets": balance_sheet.at['Assets', last_column_BS] if 'Assets' in balance_sheet.index else np.nan,
                "fy": xbrlJson['CoverPage']['DocumentFiscalYearFocus'],
                "fp": xbrlJson['CoverPage']['DocumentFiscalPeriodFocus'],
                "form": row['formType'],
                "filed": row['filedAt'],
                "Current_Assets": balance_sheet[last_column_BS].get('AssetsCurrent', np.nan),
                "Current_liabilities": balance_sheet[last_column_BS].get('LiabilitiesCurrent', np.nan),
                "Stockholder_Equity": balance_sheet[last_column_BS].get('StockholdersEquity', np.nan),
                "Liabilities_And_StockholderEquity": balance_sheet[last_column_BS].get('LiabilitiesAndStockholdersEquity', np.nan),
                "Earning_Before_Interest_And_Taxes": income_statement[last_column_IS].get('OperatingIncomeLoss', np.nan),
                "Retained_Earnings": balance_sheet[last_column_BS].get('RetainedEarningsAccumulatedDeficit', np.nan),
                "Revenues": income_statement[last_column_IS].get('Revenues',
                                                                 income_statement[last_column_IS].get('SalesRevenueNet',
                                                                 income_statement[last_column_IS].get('RevenueFromContractWithCustomerIncludingAssessedTax', np.nan))),
                "Working_capital": convert_to_float(balance_sheet[last_column_BS].get('AssetsCurrent', np.nan)) - convert_to_float(balance_sheet[last_column_BS].get('LiabilitiesCurrent', np.nan)),
                "Liabilities": balance_sheet[last_column_BS].get('Liabilities', np.nan),
                "NetCash_OperatingActivities": cash_flows[last_column_CF].get('NetCashProvidedByUsedInOperatingActivities', 
                                                 cash_flows[last_column_CF].get('NetCashProvidedByUsedInOperatingActivitiesContinuingOperations', 
                                                 cash_flows[last_column_CF].get('CashProvidedByUsedInOperatingActivitiesDiscontinuedOperations', np.nan))),
                "NetCash_InvestingActivities": cash_flows[last_column_CF].get('NetCashProvidedByUsedInInvestingActivities', 
                                            cash_flows[last_column_CF].get('NetCashProvidedByUsedInInvestingActivitiesContinuingOperations', 
                                            cash_flows[last_column_CF].get('CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations', np.nan))),
                "NetCash_FinancingActivities": cash_flows[last_column_CF].get('NetCashProvidedByUsedInFinancingActivities', 
                                             cash_flows[last_column_CF].get('NetCashProvidedByUsedInFinancingActivitiesContinuingOperations', 
                                             cash_flows[last_column_CF].get('CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations', np.nan))),
                "Cash": balance_sheet[last_column_BS].get('Cash', 
                        balance_sheet[last_column_BS].get('CashAndCashEquivalentsAtCarryingValue', np.nan)),
                "AccountsReceivable": balance_sheet[last_column_BS].get('AccountsReceivableNetCurrent', np.nan),
                "Inventory": balance_sheet[last_column_BS].get('InventoryNet', np.nan),
                "Current_Other_Assets": balance_sheet[last_column_BS].get('OtherAssetsCurrent', np.nan),
                "Noncurrent_Assets": balance_sheet[last_column_BS].get('AssetsNoncurrent', np.nan),
                "Intangible_Assets": balance_sheet[last_column_BS].get('IntangibleAssetsNetExcludingGoodwill', np.nan),
                "AccountsPayable": balance_sheet[last_column_BS].get('AccountsPayableCurrent', np.nan),
                "NetIncome": income_statement[last_column_IS].get('NetIncomeLoss', np.nan),
                "GrossProfit": income_statement[last_column_IS].get('GrossProfit', np.nan),
                "Operating_Expenses": income_statement[last_column_IS].get('OperatingExpenses', np.nan),
                "Nonoperating_Income": income_statement[last_column_IS].get('NonoperatingIncomeExpense', np.nan),
                "InterestExpense": income_statement[last_column_IS].get('InterestExpense', 
                                    income_statement[last_column_IS].get('InterestAndDebtExpense', np.nan)),
                "ShortTerm_Debt": balance_sheet[last_column_BS].get('DebtCurrent', 
                                  balance_sheet[last_column_BS].get('LongTermDebtAndCapitalLeaseObligationsCurrent', 
                                  balance_sheet[last_column_BS].get('LongTermDebtCurrent', 
                                  balance_sheet[last_column_BS].get('FinanceLeaseLiabilityCurrent', np.nan)))),
                "LongTerm_Debt": balance_sheet[last_column_BS].get('LongTermDebtAndCapitalLeaseObligations', 
                                 balance_sheet[last_column_BS].get('LongTermDebtNoncurrent', 
                                 balance_sheet[last_column_BS].get('FinanceLeaseLiabilityNoncurrent', 
                                 balance_sheet[last_column_BS].get('OperatingLeaseLiabilityNoncurrent', np.nan)))),
                "Noncurrent_Liabilities": balance_sheet[last_column_BS].get('LiabilitiesNoncurrent', np.nan),
                
                
            }
            
            # Append the data to the CSV file
            data_df = pd.DataFrame(data, index=[0])
            data_df.to_csv(csv_file, mode='a', header=False, index=False)
            
        except KeyError as e:
            print(f"KeyError: {e}. Skipping this row.")
        except Exception as e:
            print(f"An error occurred: {e}. Skipping this row.")
